In [1]:
# for capstone project

In [137]:
import pandas as pd
import numpy as np
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

In [41]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\dzarpen1\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0




geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.50   | 34 KB     | ####7      |  47% 
geographiclib-1.50   | 34 KB     | ########## | 100% 

geopy-1.21.0         | 58 KB     |            |   0% 


In [34]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')
dfp = pd.read_html(url)

In [35]:
df_edited = dfp[0]
df = df_edited[['Postal code','Borough', 'Neighborhood']]
df1 = df[df.Borough != 'Not assigned']

In [37]:
geo_data = geo_data.rename(columns = {'Postal Code':'Postal code'})
geo_data.head(2)

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


In [39]:
df2 = df1.merge(geo_data, on = 'Postal code', how = 'left')
df2

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509


In [45]:
nyork_data = df2[df2['Borough'] == 'North York'].reset_index(drop = True)
nyork_data.shape

(24, 5)

In [56]:
latitude = nyork_data['Latitude']
longitude = nyork_data['Longitude']

In [57]:
latitude_n = 43.76681
longitude_n = (-79.4163)

In [59]:
nyork_map = folium.Map(location = [latitude_n, longitude_n], zoom_start = 12)
for lat, lng, label in zip(nyork_data['Latitude'], nyork_data['Longitude'], nyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(nyork_map)  
    
nyork_map

In [70]:
CLIENT_ID = 'JFJSB5K45XHMFNY44DLYJ0OQ4GAYYCDQOQM2HGYFYIE3SVOD'
CLIENT_SECRET = 'E0O5PFWYNXB1ZTGZLHAC1LRZFFQJHBMDR2TPKQPAW5APUSRM'
VERSION = '20200404'
LIMIT = '100'

In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [76]:
nyork_venues = getNearbyVenues(names=nyork_data['Neighborhood'],
                                latitudes=nyork_data['Latitude'],
                                longitudes=nyork_data['Longitude'],)

Parkwoods
Victoria Village
Lawrence Manor / Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Fairview / Henry Farm / Oriole
Northwood Park / York University
Bayview Village
Downsview
York Mills / Silver Hills
Downsview
North Park / Maple Leaf Park / Upwood Park
Humber Summit
Willowdale / Newtonbrook
Downsview
Bedford Park / Lawrence Manor East
Humberlea / Emery
Willowdale
Downsview
York Mills West
Willowdale


In [78]:
nyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [79]:
nyork_onehot = pd.get_dummies(nyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyork_onehot['Neighborhood'] = nyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyork_onehot.columns[-1]] + list(nyork_onehot.columns[:-1])
nyork_onehot = nyork_onehot[fixed_columns]

nyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
nyork_grouped = nyork_onehot.groupby('Neighborhood').mean().reset_index()
nyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Bathurst Manor / Wilson Heights / Downsview North,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.105263,0.000000,...,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bedford Park / Lawrence Manor East,0.000000,0.000000,0.038462,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.076923,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.000,0.074074,0.037037,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.076923,0.000000,0.000,0.000000,0.076923,0.000000,0.076923,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Fairview / Henry Farm / Oriole,0.000000,0.000000,0.014925,0.000,0.014925,0.000000,0.029851,0.029851,0.014925,...,0.014925,0.000000,0.029851,0.000000,0.014925,0.014925,0.014925,0.000000,0.000000,0.029851
6,Glencairn,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Humberlea / Emery,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [81]:
num_top_venues = 3

for hood in nyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = nyork_grouped[nyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor / Wilson Heights / Downsview North----
           venue  freq
0    Coffee Shop  0.11
1           Bank  0.11
2  Shopping Mall  0.05


----Bayview Village----
                venue  freq
0  Chinese Restaurant  0.25
1                Bank  0.25
2                Café  0.25


----Bedford Park / Lawrence Manor East----
            venue  freq
0  Sandwich Place  0.08
1      Restaurant  0.08
2     Pizza Place  0.08


----Don Mills----
              venue  freq
0        Restaurant  0.07
1  Asian Restaurant  0.07
2               Gym  0.07


----Downsview----
           venue  freq
0  Grocery Store  0.23
1           Park  0.15
2           Bank  0.08


----Fairview / Henry Farm / Oriole----
                  venue  freq
0        Clothing Store  0.15
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.06


----Glencairn----
        venue  freq
0  Playground  0.25
1      Bakery  0.25
2         Pub  0.25


----Hillcrest Village----
                      venue  freq
0           

In [82]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [166]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nyork_grouped['Neighborhood']

for ind in np.arange(nyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Supermarket,Pizza Place,Bridal Shop,Diner,Pharmacy,Sandwich Place,Deli / Bodega,Shopping Mall
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Electronics Store,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
2,Bedford Park / Lawrence Manor East,Restaurant,Sandwich Place,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Locksmith,Liquor Store,Juice Bar,Café
3,Don Mills,Restaurant,Beer Store,Asian Restaurant,Coffee Shop,Japanese Restaurant,Gym,Italian Restaurant,Café,Caribbean Restaurant,Gym / Fitness Center
4,Downsview,Grocery Store,Park,Liquor Store,Gym / Fitness Center,Baseball Field,Bank,Construction & Landscaping,Athletics & Sports,Food Truck,Airport


In [167]:
# set number of clusters
kclusters = 10

nyork_grouped_clustering = nyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 4, 3, 3, 0, 3, 9, 8, 6, 2])

In [168]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
nyork_merged = nyork_data
nyork_merged = nyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nyork_merged1 = nyork_merged.drop(nyork_merged.index[16])

In [169]:
nyork_merged1

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,7.0,Park,Food & Drink Shop,Fireworks Store,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Coffee Shop,Grocery Store,Hockey Arena,Portuguese Restaurant,Nail Salon,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,3.0,Clothing Store,Women's Store,Shoe Store,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop,Accessories Store
3,M3B,North York,Don Mills,43.745906,-79.352188,3.0,Restaurant,Beer Store,Asian Restaurant,Coffee Shop,Japanese Restaurant,Gym,Italian Restaurant,Café,Caribbean Restaurant,Gym / Fitness Center
4,M6B,North York,Glencairn,43.709577,-79.445073,9.0,Japanese Restaurant,Playground,Bakery,Pub,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
5,M3C,North York,Don Mills,43.725900,-79.340923,3.0,Restaurant,Beer Store,Asian Restaurant,Coffee Shop,Japanese Restaurant,Gym,Italian Restaurant,Café,Caribbean Restaurant,Gym / Fitness Center
6,M2H,North York,Hillcrest Village,43.803762,-79.363452,8.0,Dog Run,Golf Course,Pool,Mediterranean Restaurant,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
7,M3H,North York,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259,3.0,Bank,Coffee Shop,Supermarket,Pizza Place,Bridal Shop,Diner,Pharmacy,Sandwich Place,Deli / Bodega,Shopping Mall
8,M2J,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556,3.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Bakery,Bank,Tea Room,Food Court,Convenience Store,Electronics Store
9,M3J,North York,Northwood Park / York University,43.767980,-79.487262,3.0,Coffee Shop,Metro Station,Caribbean Restaurant,Massage Studio,Bar,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


In [172]:
# create map
map_clusters = folium.Map(location=[latitude_n, longitude_n], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyork_merged1['Latitude'], nyork_merged1['Longitude'], nyork_merged1['Neighborhood'], nyork_merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [173]:
nyork_merged.loc[nyork_merged['Cluster Labels'] == 0, nyork_merged.columns[[1] + list(range(5, nyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,North York,0.0,Grocery Store,Park,Liquor Store,Gym / Fitness Center,Baseball Field,Bank,Construction & Landscaping,Athletics & Sports,Food Truck,Airport
13,North York,0.0,Grocery Store,Park,Liquor Store,Gym / Fitness Center,Baseball Field,Bank,Construction & Landscaping,Athletics & Sports,Food Truck,Airport
14,North York,0.0,Construction & Landscaping,Bakery,Park,Women's Store,Dog Run,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
17,North York,0.0,Grocery Store,Park,Liquor Store,Gym / Fitness Center,Baseball Field,Bank,Construction & Landscaping,Athletics & Sports,Food Truck,Airport
21,North York,0.0,Grocery Store,Park,Liquor Store,Gym / Fitness Center,Baseball Field,Bank,Construction & Landscaping,Athletics & Sports,Food Truck,Airport


In [174]:
nyork_merged.loc[nyork_merged['Cluster Labels'] == 1, nyork_merged.columns[[1] + list(range(5, nyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,1.0,Park,Cafeteria,Women's Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [175]:
nyork_merged.loc[nyork_merged['Cluster Labels'] == 2, nyork_merged.columns[[1] + list(range(5, nyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,2.0,Food Service,Baseball Field,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [176]:
nyork_merged.loc[nyork_merged['Cluster Labels'] == 3, nyork_merged.columns[[1] + list(range(5, nyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3.0,Coffee Shop,Grocery Store,Hockey Arena,Portuguese Restaurant,Nail Salon,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,North York,3.0,Clothing Store,Women's Store,Shoe Store,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Vietnamese Restaurant,Miscellaneous Shop,Accessories Store
3,North York,3.0,Restaurant,Beer Store,Asian Restaurant,Coffee Shop,Japanese Restaurant,Gym,Italian Restaurant,Café,Caribbean Restaurant,Gym / Fitness Center
5,North York,3.0,Restaurant,Beer Store,Asian Restaurant,Coffee Shop,Japanese Restaurant,Gym,Italian Restaurant,Café,Caribbean Restaurant,Gym / Fitness Center
7,North York,3.0,Bank,Coffee Shop,Supermarket,Pizza Place,Bridal Shop,Diner,Pharmacy,Sandwich Place,Deli / Bodega,Shopping Mall
8,North York,3.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Bakery,Bank,Tea Room,Food Court,Convenience Store,Electronics Store
9,North York,3.0,Coffee Shop,Metro Station,Caribbean Restaurant,Massage Studio,Bar,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
18,North York,3.0,Restaurant,Sandwich Place,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Locksmith,Liquor Store,Juice Bar,Café
20,North York,3.0,Coffee Shop,Ramen Restaurant,Pizza Place,Restaurant,Grocery Store,Sandwich Place,Café,Sushi Restaurant,Pet Store,Plaza
23,North York,3.0,Coffee Shop,Ramen Restaurant,Pizza Place,Restaurant,Grocery Store,Sandwich Place,Café,Sushi Restaurant,Pet Store,Plaza
